# Project Title:- Agentic AI-Based Travel Planning System Using LangChain

# Project Overview:
Planning a trip involves selecting flights, hotels, and Locations to visit while considering constraints such as budget, travel duration, weather, and personal preferences. Currently, this process requires users to search across multiple platforms, compare information manually, and create itineraries that may be inefficient or incomplete.

# Project Objective:
The objective of this project is to build an agentic AI-based travel planning system that can automatically generate personalized and realistic itineraries. The system is designed to understand user input in natural language and perform multi-step reasoning to make informed decisions, similar to how a human travel planner would operate.

# Project Technicality:
From a technical standpoint, the system must integrate structured travel data (flights, hotels, and attractions) with real-time weather information. It should filter, rank, and select the best options based on user constraints and combine them into a clear day-wise itinerary. Additionally, the system should provide basic explainability by justifying key selections such as flight and accomadation choices.

# Stake Holders:
    • End Users (Travelers): Want fast, affordable, and personalized trip plans
    • Travel Companies: Aim to automate planning and reduce support workload
    • Travel Agents: Benefit from AI-assisted itinerary generation
    • Platform Providers: Improve user experience using conversational AI


# Project Requirements:
    • Natural Language Understanding: Accept travel queries in plain English (destination, dates, budget, interests)
    • Multi-Step Decision Making: Select flights, hotels, and attractions sequentially. Combine results into a coherent itinerary
    • Tool-Based Information Retrieval: Fetch flight, hotel, and attraction data from structured datasets. Retrieve real-time weather            data using an external API 
    • Personalization: Tailor recommendations based on budget, trip duration, and interests

# System Design:
The system follows a modular and agent-based architecture so that each part has a clear role and can be easily updated.

A User Interface layer (Streamlit or CLI) is used to take travel details from the user and display the final itinerary.

The Agent layer (built using LangChain) acts as the brain of the system. It understands the user request, decides the steps to follow, and controls the overall workflow.

The Tool layer contains separate tools for flights, hotels, places, weather, and budget calculation. Each tool performs one specific task.

The Data layer provides information to the tools using local JSON files and a real-time weather API.

The LLM layer handles language understanding and reasoning to generate meaningful and structured travel plans.

This architecture allows the system to automatically decide what information to fetch and how to combine it into a complete itinerary.

# STEP 1:- TOOL DEVELOPMENT

# Flight Search Tool
This Tool Filters the Flights from the given data based on Lower Price and Shorter Duration

In [1]:
"""
Flight Search Tool Module

This module loads flight data from a JSON file, filters flights
based on user input, selects the best flight according to preference,
and exposes a LangChain-compatible flight search tool.
"""

import json
from datetime import datetime
from langchain.tools import tool


# ----------------------------------
# Data Loading
# ----------------------------------
def load_flights():
    """
    Load flight data from a JSON file.

    Returns:
        list: List of flight records.
    """
    with open("flights.json", "r") as file:
        return json.load(file)


# ----------------------------------
# Flight Filtering
# ----------------------------------
def filter_flights(source: str, destination: str):
    """
    Filter flights based on source and destination cities.

    Arguments of the Function:
        source (str): Source city.
        destination (str): Destination city.

    Returns:
        list: Matching flight records.
    """
    flights = load_flights()

    return [
        flight
        for flight in flights
        if flight["from"].strip().lower() == source.strip().lower()
        and flight["to"].strip().lower() == destination.strip().lower()
    ]


# ----------------------------------
# Duration Calculation
# ----------------------------------
def calculate_duration(departure_time: str, arrival_time: str) -> float:
    """
    Calculate flight duration in minutes.

    Arguments of the Function:
        departure_time (str): ISO formatted departure time.
        arrival_time (str): ISO formatted arrival time.

    Returns:
        float: Duration in minutes.
    """
    departure = datetime.fromisoformat(departure_time)
    arrival = datetime.fromisoformat(arrival_time)

    duration = arrival - departure
    return duration.total_seconds() / 60


# ----------------------------------
# Flight Selection Logic
# ----------------------------------
def select_best_flight(flights: list, preference: str = "cheapest"):
    """
    Select the best flight based on user preference.

    Arguments of the Function:
        flights (list): List of available flights.
        preference (str): 'cheapest' or 'fastest'.

    Returns:
        dict | None: Best flight or None if no flights found.
    """
    if not flights:
        return None

    if preference == "fastest":
        return min(
            flights,
            key=lambda flight: calculate_duration(
                flight["departure_time"],
                flight["arrival_time"]
            )
        )

    return min(flights, key=lambda flight: flight["price"])


# ----------------------------------
# LangChain Tool
# Langchain enables the AI to think step by step and Use tools to perform tasks
# @tool is the decorator used to wrap the function so that the Agent can call the tool
# ----------------------------------
@tool
def flight_search_tool(source: str, destination: str, preference: str):
    """
    Find the best flight given source, destination,
    and user preference (cheapest or fastest).

    Arguments of the Function:
        source (str): Source city.
        destination (str): Destination city.
        preference (str): 'cheapest' or 'fastest'.

    Returns:
        dict: Structured flight result or failure message.
    """
    flights = filter_flights(source, destination)
    best_flight = select_best_flight(flights, preference)

    if not best_flight:
        return {
            "message": f"No flights found from {source} to {destination}."
        }

    duration_minutes = calculate_duration(
        best_flight["departure_time"],
        best_flight["arrival_time"]
    )

    return {
        "flight": {
            "airline": best_flight["airline"],
            "price": best_flight["price"],
            "duration_minutes": duration_minutes,
            "departure_time": best_flight["departure_time"]
        },
        "message": (
            f"Best flight from {source} to {destination}: "
            f"{best_flight['airline']} at ${best_flight['price']}."
        )
    }


# ----------------------------------
# Local Test Execution
# This function for testing will only be executed directly not when the file is imported
# This step is important because, during the streamlit app development, there is no need for the execution of this function
# ----------------------------------
if __name__ == "__main__":
    output = flight_search_tool.invoke({
        "source": "Hyderabad",
        "destination": "Delhi",
        "preference": "fastest"
    })

    print(output)


{'flight': {'airline': 'IndiGo', 'price': 2907, 'duration_minutes': 240.0, 'departure_time': '2025-01-04T11:32:00'}, 'message': 'Best flight from Hyderabad to Delhi: IndiGo at $2907.'}


# Hotel Recommendation Tool

In [2]:
import json
from langchain.tools import tool


# ----------------------------------
# Data Loading Functions
# ----------------------------------
def load_hotels():
    """
    Loads hotel data from the hotels.json file.

    Returns:
        list: A list of hotel records.
    """
    with open("hotels.json", "r") as file:
        return json.load(file)


# ----------------------------------
# Hotel Filtering Logic
# ----------------------------------
def filter_hotels(city, rating, price):
    """
    Filters hotels based on city, minimum rating, and maximum price.

    Args:
        city (str): City name
        rating (int): Minimum hotel rating
        price (int): Maximum price per night

    Returns:
        list: Filtered list of hotels
    """
    hotels = load_hotels()

    return [
        hotel
        for hotel in hotels
        if hotel["city"].lower() == city.lower()
        and int(hotel["stars"]) >= rating
        and int(hotel["price_per_night"]) <= price
    ]


# ----------------------------------
# Hotel Ranking Logic
# ----------------------------------
def rank_hotels(hotels, preference="cheapest"):
    """
    Selects the best hotel based on user preference.

    Args:
        hotels (list): List of filtered hotels
        preference (str): 'cheapest' or 'highest_rating'

    Returns:
        dict or None: Best hotel or None if list is empty
    """
    if not hotels:
        return None

    if preference == "highest_rating":
        return max(hotels, key=lambda hotel: hotel["stars"])

    return min(hotels, key=lambda hotel: hotel["price_per_night"])


# ----------------------------------
# LangChain Tool Definition
# ----------------------------------
@tool
def hotel_search_tool(city: str, price: int, rating: int, preference: str):
    """
    Finds the best hotel in a city based on price, rating, and preference.

    Args:
        city (str): Destination city
        price (int): Maximum price per night
        rating (int): Minimum hotel rating
        preference (str): 'cheapest' or 'highest_rating'

    Returns:
        dict: Structured hotel search result
    """
    filtered_hotels = filter_hotels(city, rating, price)
    best_hotel = rank_hotels(filtered_hotels, preference)

    if not best_hotel:
        return {
            "message": f"No hotels found in {city} within price ${price} and rating {rating}+."
        }

    return {
        "hotel": {
            "name": best_hotel["name"],
            "price": best_hotel["price_per_night"],
            "rating": best_hotel["stars"],
            "amenities": best_hotel["amenities"],
        },
        "message": (
            f"Best hotel in {city}: "
            f"{best_hotel['name']} at ${best_hotel['price_per_night']} per night."
        ),
    }


# ----------------------------------
# Tool Testing (Runs Only When Executed Directly)
# ----------------------------------
if __name__ == "__main__":
    test_output = hotel_search_tool.run(
        {
            "city": "Delhi",
            "price": 4783,
            "rating": 2,
            "preference": "cheapest",
        }
    )

    print(test_output)


{'hotel': {'name': 'Blue Lagoon Resort', 'price': 2743, 'rating': 2, 'amenities': ['gym', 'breakfast']}, 'message': 'Best hotel in Delhi: Blue Lagoon Resort at $2743 per night.'}


# Location Discovery Tool

In [3]:
import json
from langchain.tools import tool


# ----------------------------------
# Data Loading
# ----------------------------------
def load_places():
    """
    Loads the places.json file which contains location data.

    Returns:
        list: A list of place records.
    """
    with open("places.json", "r") as file:
        return json.load(file)


# ----------------------------------
# Location Filtering
# ----------------------------------
def filter_locations(city, category):
    """
    Filters locations based on city and category.

    Args:
        city (str): City name
        category (str): Type of location (e.g., market, beach, temple)

    Returns:
        list: Filtered list of locations
    """
    places = load_places()

    return [
        place
        for place in places
        if place["city"].strip().lower() == city.strip().lower()
        and place["type"].strip().lower() == category.strip().lower()
    ]


# ----------------------------------
# Ranking Logic
# ----------------------------------
def rank_places(places):
    """
    Ranks places based on higher rating.

    Args:
        places (list): List of filtered places

    Returns:
        list: Places sorted by rating (highest first)
    """
    if not places:
        return []

    return sorted(places, key=lambda place: place["rating"], reverse=True)


# ----------------------------------
# Limiting Number of Places
# ----------------------------------
def max_places(places, max_days):
    """
    Limits the number of places based on number of days.

    Args:
        places (list): Ranked places
        max_days (int): Maximum days available

    Returns:
        list: Limited list of places
    """
    return places[:max_days]


# ----------------------------------
# Selecting Required Information
# ----------------------------------
def select_info(places):
    """
    Extracts required fields from places.

    Args:
        places (list): List of places

    Returns:
        list: Simplified place information
    """
    return [
        {
            "name": place["name"],
            "category": place["type"],
            "rating": place["rating"],
        }
        for place in places
    ]


# ----------------------------------
# Day-wise Grouping
# ----------------------------------
def group_places_by_day(places, days):
    """
    Groups places into a day-wise itinerary.

    Args:
        places (list): Selected places
        days (int): Number of days

    Returns:
        dict: Day-wise itinerary
    """
    itinerary = {f"Day {i + 1}": [] for i in range(days)}

    for index, place in enumerate(places):
        day_key = f"Day {(index % days) + 1}"
        itinerary[day_key].append(place)

    return itinerary


# ----------------------------------
# LangChain Tool
# ----------------------------------
@tool
def location_search_tool(city: str, category: str, max_days: int):
    """
    Shows the maximum number of locations that can be visited
    based on city, category, and number of days.

    Args:
        city (str): City name
        category (str): Location category
        max_days (int): Number of days

    Returns:
        dict: Day-wise itinerary
    """
    filtered_places = filter_locations(city, category)
    ranked_places = rank_places(filtered_places)
    limited_places = max_places(ranked_places, max_days)
    selected_places = select_info(limited_places)
    day_wise_itinerary = group_places_by_day(selected_places, max_days)

    return {
        "itinerary": day_wise_itinerary
    }


# ----------------------------------
# Tool Testing (Safe Execution)
# ----------------------------------
if __name__ == "__main__":
    test_output = location_search_tool.run(
        {
            "city": "Goa",
            "category": "market",
            "max_days": 3,
        }
    )

    print(test_output)


{'itinerary': {'Day 1': [{'name': 'Beautiful Lake', 'category': 'market', 'rating': 4.0}], 'Day 2': [], 'Day 3': []}}


# Weather Lookup Tool

In [4]:
import requests
from langchain.tools import tool


# ----------------------------------
# Geocoding: City → Coordinates
# ----------------------------------
def get_city_coordinates(city):
    """
    Uses the Open-Meteo Geocoding API to fetch latitude and longitude
    for the given city name.

    Args:
        city (str): Name of the city

    Returns:
        tuple | None: (latitude, longitude) if found, else None
    """
    url = "https://geocoding-api.open-meteo.com/v1/search"
    params = {
        "name": city,
        "count": 1  # Only return the best matching city
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "results" not in data:
        return None

    location = data["results"][0]
    return location["latitude"], location["longitude"]


# ----------------------------------
# Weather API Fetch
# ----------------------------------
def fetch_weather(lat, lon, start_date, end_date):
    """
    Fetches daily weather forecast from Open-Meteo API
    for given coordinates and date range.

    Args:
        lat (float): Latitude
        lon (float): Longitude
        start_date (str): Start date (YYYY-MM-DD)
        end_date (str): End date (YYYY-MM-DD)

    Returns:
        dict: Raw weather data from API
    """
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": [
            "temperature_2m_max",
            "temperature_2m_min",
            "weathercode"
        ],
        "timezone": "auto"
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data


# ----------------------------------
# Weather Code Loader
# ----------------------------------
def load_weather_codes(file_path):
    """
    Loads weather codes and their descriptions
    from a text file.

    File format:
        code=description

    Args:
        file_path (str): Path to weather code file

    Returns:
        dict: Mapping of weather code to description
    """
    codes = {}

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if not line:
                continue

            try:
                code, description = line.split("=")
                codes[int(code.strip())] = description.strip()
            except ValueError:
                print(f"Skipping invalid line: {line}")

    return codes


# ----------------------------------
# Load Weather Codes Once
# ----------------------------------
WEATHER_CODES = load_weather_codes("weather.txt")


# ----------------------------------
# Weather Formatting
# ----------------------------------
def format_weather(data):
    """
    Formats raw weather data into a clean daily forecast.

    Args:
        data (dict): Raw weather data from API

    Returns:
        list: Daily weather forecast
    """
    if "daily" not in data:
        return None

    daily_forecast = []

    times = data["daily"].get("time", [])
    max_temps = data["daily"].get("temperature_2m_max", [])
    min_temps = data["daily"].get("temperature_2m_min", [])
    weather_codes = data["daily"].get("weathercode", [])

    for i in range(len(times)):
        daily_forecast.append({
            "date": times[i],
            "condition": WEATHER_CODES.get(weather_codes[i], "Unknown"),
            "temp_range": f"{min_temps[i]}–{max_temps[i]} °C"
        })

    return daily_forecast


# ----------------------------------
# LangChain Tool Wrapper
# ----------------------------------
@tool
def weather_lookup_tool(city: str, start_date: str, end_date: str):
    """
    Provides daily weather forecast for a city
    between given start and end dates.

    Args:
        city (str): City name
        start_date (str): Start date (YYYY-MM-DD)
        end_date (str): End date (YYYY-MM-DD)

    Returns:
        list | dict: Daily weather forecast or error message
    """
    coordinates = get_city_coordinates(city)

    if not coordinates:
        return {"message": "City not found"}

    latitude, longitude = coordinates
    raw_weather_data = fetch_weather(
        latitude,
        longitude,
        start_date,
        end_date
    )

    formatted_weather = format_weather(raw_weather_data)
    if not formatted_weather:
        return {"message": "Weather data not available for the selected dates."}
    return formatted_weather


# ----------------------------------
# Tool Testing
# ----------------------------------
if __name__ == "__main__":
    test_result = weather_lookup_tool.run(
        {
            "city": "Delhi",
            "start_date": "2025-12-17",
            "end_date": "2025-12-19"
        }
    )

    print(test_result)

[{'date': '2025-12-17', 'condition': '"Mainly clear",', 'temp_range': '11.2–22.0 °C'}, {'date': '2025-12-18', 'condition': '"Fog",', 'temp_range': '10.6–20.7 °C'}, {'date': '2025-12-19', 'condition': '"Fog",', 'temp_range': '11.3–20.2 °C'}]


# Budget Estimation Tool

In [5]:
from langchain.tools import tool


# ----------------------------------
# Budget Calculation Logic
# ----------------------------------
def calculate_budget(
    flight_price: int,
    hotel_price_per_night: int,
    number_of_days: int,
    daily_expense: int = 1500
):
    """
    Calculates the total trip budget.

    Args:
        flight_price (int): Cost of the flight
        hotel_price_per_night (int): Cost per night of the hotel
        number_of_days (int): Total number of travel days
        daily_expense (int, optional): Daily local expenses. Defaults to 1500.

    Returns:
        dict: Breakdown of total trip expenses
    """
    # Number of nights stayed at the hotel
    hotel_nights = number_of_days - 1

    # Total accommodation cost
    hotel_cost = hotel_price_per_night * hotel_nights

    # Total daily local expenses
    local_expenses = daily_expense * number_of_days

    # Final total trip cost
    total_cost = flight_price + hotel_cost + local_expenses

    return {
        "flight_cost": flight_price,
        "hotel_cost": hotel_cost,
        "local_expenses": local_expenses,
        "total_cost": total_cost
    }


# ----------------------------------
# LangChain Budget Tool Wrapper
# ----------------------------------
@tool
def budget_estimation_tool(
    flight_price: int,
    hotel_price_per_night: int,
    number_of_days: int
):
    """
    Calculates total trip budget including flight,
    hotel accommodation, and daily expenses.

    Args:
        flight_price (int): Flight ticket cost
        hotel_price_per_night (int): Hotel price per night
        number_of_days (int): Total trip duration

    Returns:
        dict: Budget breakdown including total cost
    """
    daily_expense = 1500

    hotel_nights = number_of_days - 1
    hotel_cost = hotel_price_per_night * hotel_nights
    local_expenses = daily_expense * number_of_days
    total_cost = flight_price + hotel_cost + local_expenses

    return {
        "flight_cost": flight_price,
        "hotel_cost": hotel_cost,
        "local_expenses": local_expenses,
        "total_cost": total_cost
    }


# ----------------------------------
# Tool Testing
# ----------------------------------
if __name__ == "__main__":
    output = budget_estimation_tool.run({
        "flight_price": 6500,
        "hotel_price_per_night": 3000,
        "number_of_days": 3
    })

    print(output)

{'flight_cost': 6500, 'hotel_cost': 6000, 'local_expenses': 4500, 'total_cost': 17000}


# STEP2:- AGENT DESIGNING
The Travel Agent understands the user requests.
It breaks the request into small tasks like finding flights, hotels, places, weather, and budget.
The agent automatically decides which tools to use and collects results from them.
Finally, it combines everything and gives the best travel plan like a human travel expert.

The agent reads the user’s travel request written in natural language.
It automatically identifies important details like source city, destination, dates, and budget.
It also understands user interests such as beaches, culture, or food, and the trip duration.
This extraction is done by the AI itself, not by writing separate parsing code.

We use a LangChain Tool-Calling Agent to build the system.
This type of agent knows when and how to call different tools automatically.
It is more stable and easier to use, especially for beginners.

In [13]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate

# ----------------------------------
# Large Language Model Configuration
# ----------------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini",  # OpenAI model used for reasoning and tool calling
    temperature=0        # Keeps responses deterministic and factual
)


# ----------------------------------
# Agent Prompt Definition
# ----------------------------------
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
You are a deterministic travel planning assistant.

STRICT RULES:
1. You MUST use tools for all factual data.
2. You MUST NOT guess, infer, or recommend alternatives unless explicitly requested.
3. If any required tool returns a failure message:
   - STOP execution immediately
   - RETURN a structured JSON response explaining the failure
4. Final output MUST be in valid JSON followed by a short readable summary.
5. Budget calculations MUST use tool outputs only.
6. If planning cannot be completed, ask the user to modify constraints

REQUIRED OUTPUT FORMAT:
{{
  "status": "success | failed",
  "data": {{}},
  "message": "human-readable explanation"
}}
"""
    ),
    ("human", "{input}"), # Place Holder for the user input
    ("assistant", "{agent_scratchpad}") # stores the tool calls and reasoning
])


# ----------------------------------
# Tool Registration
# ----------------------------------
tools = [
    flight_search_tool,
    hotel_search_tool,
    weather_lookup_tool,
    budget_estimation_tool
]


# ----------------------------------
# Agent Creation
# ----------------------------------
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)


# ----------------------------------
# Agent Executor
# ----------------------------------
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True  # Displays tool calls and reasoning steps
)


# ----------------------------------
# User Input & Execution
# ----------------------------------
def run_travel_agent():
    """
    Accepts user input from the terminal and executes
    the agent to generate a structured travel plan.
    """
    user_input = input("Enter your travel request: ")

    response = agent_executor.invoke({
        "input": user_input
    })

    print(response)


# ----------------------------------
# Entry Point
# ----------------------------------
if __name__ == "__main__":
    run_travel_agent()

Enter your travel request:  Plan a 3-day trip from Hyderabad to Mumbai using the cheapest flight, a hotel under 2500 per night with 3-star rating, including weather forecast and total budget.




> Entering new AgentExecutor chain...

Invoking: `flight_search_tool` with `{'source': 'Hyderabad', 'destination': 'Mumbai', 'preference': 'cheapest'}`


{'flight': {'airline': 'Go First', 'price': 6762, 'duration_minutes': 180.0, 'departure_time': '2025-02-24T07:40:00'}, 'message': 'Best flight from Hyderabad to Mumbai: Go First at $6762.'}
Invoking: `hotel_search_tool` with `{'city': 'Mumbai', 'price': 2500, 'rating': 3, 'preference': 'cheapest'}`


{'message': 'No hotels found in Mumbai within price $2500 and rating 3+.'}
Invoking: `weather_lookup_tool` with `{'city': 'Mumbai', 'start_date': '2023-10-01', 'end_date': '2023-10-03'}`


{'message': 'Weather data not available for the selected dates.'}
Invoking: `flight_search_tool` with `{'source': 'Hyderabad', 'destination': 'Mumbai', 'preference': 'cheapest'}`


{'flight': {'airline': 'Go First', 'price': 6762, 'duration_minutes': 180.0, 'departure_time': '2025-02-24T07:40:00'}, 'message': 'Best flight from Hyderabad to Mumbai: Go 

# Summary
The agent acts as an intelligent travel planner, interpreting user requests and deciding which tools to call. It orchestrates multiple tools—flight, hotel, places, weather, and budget—to gather real-time data and generate a complete itinerary. The reasoning strategy is step-by-step: understand the request, call tools, combine results, and produce structured output. The system uses LangChain’s Tool-Calling paradigm, conceptually similar to ReAct, to enable multi-step, tool-assisted reasoning. All outputs are structured and human-readable, ensuring clarity and reliability.